# Model

## Libs

In [1]:
import gc
import json
import pandas as pd
import unicodedata
import scipy.sparse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
import joblib
import os
import time

## Data

In [2]:
Data_path = '../data/json/'

In [3]:
with open(Data_path + 'applicants.json', 'r', encoding='utf-8') as file:
	applicants = json.load(file)

rows = []
for applicant in applicants:
	id = applicant
	informacoes_pessoais = applicants.get(id).get("informacoes_pessoais")
	informacoes_profissionais = applicants.get(id).get("informacoes_profissionais")
	formacao_e_idiomas = applicants.get(id).get("formacao_e_idiomas")
	aux = {
		'id': int(id),
		'pcd': informacoes_pessoais.get('pcd'),
		'titulo_profissional': informacoes_profissionais.get('titulo_profissional'),
		'area_atuacao': informacoes_profissionais.get('area_atuacao'),
		'conhecimentos_tecnicos': informacoes_profissionais.get('conhecimentos_tecnicos'),
		'certificacoes': informacoes_profissionais.get('certificacoes'),
		'outras_certificacoes': informacoes_profissionais.get('outras_certificacoes'),
		'nivel_academico': formacao_e_idiomas.get('nivel_academico'),
		'nivel_ingles': formacao_e_idiomas.get('nivel_ingles'),
		'nivel_espanhol': formacao_e_idiomas.get('nivel_espanhol'),
		'outro_idioma': formacao_e_idiomas.get('outro_idioma'),
		'cv_pt': applicants.get(id).get("cv_pt"),
	}
	rows.append(aux)

applicants_df = pd.DataFrame(data=rows, columns=['id', 'pcd', 'endereco', 'titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos','certificacoes', 'outras_certificacoes', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'cv_pt'])

In [4]:
with open(Data_path + 'vagas.json', 'r', encoding='utf-8') as file:
	vagas = json.load(file)

rows = []
for vaga in vagas:
	id = vaga
	perfil_vaga = vagas.get(id).get("perfil_vaga")
	aux = {
		'id': int(id),
		'vaga_especifica_para_pcd': perfil_vaga.get('vaga_especifica_para_pcd'),
		'nivel_profissional': perfil_vaga.get('nivel profissional'),
		'nivel_academico': perfil_vaga.get('nivel_academico'),
		'nivel_ingles': perfil_vaga.get('nivel_ingles'),
		'nivel_espanhol': perfil_vaga.get('nivel_espanhol'),
		'outro_idioma': perfil_vaga.get('outro_idioma'),
		'areas_atuacao': perfil_vaga.get('areas_atuacao'),
		'principais_atividades': perfil_vaga.get('principais_atividades'),
		'demais_observacoes': perfil_vaga.get('demais_observacoes'),
		'viagens_requeridas': perfil_vaga.get('viagens_requeridas')
	}
	rows.append(aux)

vagas_df = pd.DataFrame(data=rows, columns=['id', 'estado', 'vaga_especifica_para_pcd', 'nivel_profissional', 'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma', 'areas_atuacao', 'principais_atividades', 'demais_observacoes', 'viagens_requeridas'])

In [5]:
with open(Data_path + 'prospects.json', 'r', encoding='utf-8') as file:
	prospects = json.load(file)

rows = []
for prospect in prospects:
	id = prospect
	candidates = prospects.get(id).get("prospects")
	for candidate in candidates:
		aux_id:int = int(id)
		aux = {
			'id_vaga': aux_id,
			'id_applicant': int(candidate.get("codigo")),
			'situacao_candidado': candidate.get("situacao_candidado"),
			'comentario': candidate.get("comentario")
		}
		rows.append(aux)

prospects_df = pd.DataFrame(data=rows, columns=['id_vaga','id_applicant', 'situacao_candidado', 'comentario'])

In [6]:
aux_merge_ap = pd.merge(applicants_df, prospects_df, left_on='id', right_on='id_applicant', how='inner')
merge = pd.merge(aux_merge_ap, vagas_df, left_on='id_vaga', right_on='id', how='inner', suffixes=('', '_vaga'))

df = merge[[
    'pcd',
    'titulo_profissional',
    'area_atuacao',
    'conhecimentos_tecnicos',
    'certificacoes',
    'outras_certificacoes',
    'nivel_academico',
    'nivel_ingles',
    'nivel_espanhol',
    'outro_idioma',
    'cv_pt',
    'situacao_candidado',
    'comentario',
    'vaga_especifica_para_pcd',
    'nivel_profissional',
    'nivel_academico_vaga',
    'nivel_ingles_vaga',
    'nivel_espanhol_vaga',
    'outro_idioma_vaga',
    'areas_atuacao',
    'principais_atividades',
    'demais_observacoes',
    'viagens_requeridas'
]]

df = df.drop_duplicates()

In [7]:
del applicants
del vagas
del prospects
del applicants_df
del prospects_df
del vagas_df
del aux_merge_ap
del merge
gc.collect()

0

## Data Pre Processor

In [8]:
nivel_map = {
    'nenhum': 0,
    'basico': 1,
    'intermediario': 2,
    'avancado': 3,
    'fluente': 4
}

situacao_map = {
    "candidatura em andamento": [
        "inscrito",
        "prospect",
        "em avaliacao pelo rh",
        "entrevista tecnica",
        "entrevista com cliente",
        "encaminhado ao requisitante",
        "encaminhar proposta",
        "documentacao pj",
        "documentacao clt",
        "documentacao cooperado"
    ],
    "contratacao confirmada": [
        "aprovado",
        "proposta aceita",
        "contratado pela decision",
        "contratado como hunting"
    ],
    "recusado": [
        "nao aprovado pelo rh",
        "nao aprovado pelo requisitante",
        "nao aprovado pelo cliente"
    ],
    "desistencia": [
        "desistiu",
        "desistiu da contratacao",
        "recusado",
        "sem interesse nesta vaga"
    ]
}

In [9]:
def string_normalizer(text):
	if text is None or text == "":
		return None
	else:
		return ''.join(c for c in unicodedata.normalize('NFD', text) if not unicodedata.combining(c)).lower()

def status_map(text):
    for grupo, status_list in situacao_map.items():
        if text in status_list:
            return grupo
    return None

In [10]:
df = df.apply(lambda col: col.map(string_normalizer) if col.dtype == 'object' else col)
df.replace('-', None, inplace=True)

In [11]:
df['pcd'] = df['pcd'].apply(lambda x: 1 if x == 'sim' else 0)
df['vaga_especifica_para_pcd'] = df['vaga_especifica_para_pcd'].apply(lambda x: 1 if x == 'sim' else 0)
df['pcd_match'] = ((df['pcd'] == 1) & (df['vaga_especifica_para_pcd'] == 1)).astype(int)

In [12]:
df['viagens_requeridas'] = df['viagens_requeridas'].apply(lambda x: 1 if x == 'sim' else 0)

In [13]:
df['nivel_ingles'] = df['nivel_ingles'].map(nivel_map).fillna(0)
df['nivel_ingles_vaga'] = df['nivel_ingles_vaga'].map(nivel_map).fillna(0)
df['ingles'] = (df['nivel_ingles'] >= df['nivel_ingles_vaga']).astype(int)

In [14]:
df['nivel_espanhol'] = df['nivel_ingles'].map(nivel_map).fillna(0)
df['nivel_espanhol_vaga'] = df['nivel_ingles_vaga'].map(nivel_map).fillna(0)
df['espanhol'] = (df['nivel_espanhol'] >= df['nivel_espanhol_vaga']).astype(int)

In [15]:
df['situacao_candidado'] = df['situacao_candidado'].map(status_map)

In [16]:
df.fillna("", inplace=True)

In [17]:
candidato_cols = ['titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos', 'certificacoes', 'outras_certificacoes', 'nivel_academico', 'outro_idioma', 'cv_pt']
df['candidato'] = df[candidato_cols].agg(' '.join, axis=1)

In [18]:
vaga_cols = ['nivel_profissional', 'nivel_academico_vaga', 'outro_idioma_vaga', 'areas_atuacao', 'principais_atividades', 'demais_observacoes']
df['vaga'] = df[vaga_cols].agg(' '.join, axis=1)

In [19]:
df = df[['candidato', 'vaga', 'viagens_requeridas', 'pcd_match', 'ingles', 'espanhol', 'comentario', 'situacao_candidado']]

In [20]:
df.rename(columns={
    'situacao_candidado': 'situacao',
    'comentario': 'comentario',
    'viagens_requeridas': 'vaga_viagens_requeridas',
    'pcd_match': 'pcd',
    'ingles': 'nivel_ingles',
    'espanhol': 'nivel_espanhol'
}, inplace=True)

## Models

In [21]:
df_model = df.copy()

In [22]:
label_encoders = {}
le = LabelEncoder()
df_model['situacao'] = le.fit_transform(df_model['situacao'])
label_encoders['situacao'] = le

le.inverse_transform()

In [23]:
vectorizer_candidato = TfidfVectorizer(max_features=3000)
vt_candidato = vectorizer_candidato.fit_transform(df_model['candidato'])

In [24]:
vectorizer_vaga = TfidfVectorizer(max_features=3000)
vt_vaga = vectorizer_candidato.fit_transform(df_model['vaga'])

In [25]:
vectorizer_comentario = TfidfVectorizer(max_features=1000)
vt_comentario = vectorizer_candidato.fit_transform(df_model['comentario'])

In [26]:
aux = df_model[['vaga_viagens_requeridas', 'pcd', 'nivel_ingles', 'nivel_espanhol']]
scaler = StandardScaler()
num_scaled = scaler.fit_transform(aux)

In [27]:
X = scipy.sparse.hstack([num_scaled, vt_candidato, vt_vaga, vt_comentario])
y = df_model['situacao']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

### Random Florest

#### Model

In [28]:
rf = RandomForestClassifier(n_estimators=200, random_state=42, max_depth=20, class_weight='balanced')

#### Train

In [29]:
start_fit_rf = time.time()
rf.fit(X_train, y_train)
end_fit_rf = time.time()

#### Predict

In [30]:
start_predict_rf = time.time()
y_pred_rf = rf.predict(X_test)
end_predict_rf = time.time()

print(classification_report(y_test, y_pred_rf, zero_division=0))

              precision    recall  f1-score   support

           0       0.93      0.94      0.93      8657
           1       0.39      0.57      0.46       626
           2       0.78      0.56      0.65       617
           3       0.81      0.73      0.77      1295

    accuracy                           0.87     11195
   macro avg       0.73      0.70      0.70     11195
weighted avg       0.88      0.87      0.87     11195



### XGBoost

#### Model

In [31]:
xgb = XGBClassifier(
    objective='multi:softmax',
    num_class=len(y.unique()),
    eval_metric='mlogloss',
    random_state=42,
    n_estimators=200,
    max_depth=20,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
	min_child_weight=2
)

#### Train

In [32]:
start_fit_xgb = time.time()
xgb.fit(X_train, y_train)
end_fit_xgb = time.time()

#### Predict

In [33]:
start_predict_xgb = time.time()
y_pred_xgb = xgb.predict(X_test)
end_predict_xgb = time.time()
print(classification_report(y_test, y_pred_xgb, zero_division=0))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      8657
           1       0.77      0.38      0.51       626
           2       0.82      0.58      0.68       617
           3       0.83      0.78      0.80      1295

    accuracy                           0.91     11195
   macro avg       0.84      0.68      0.74     11195
weighted avg       0.90      0.91      0.90     11195



## Metrics

In [34]:
content = (f'Random Forest Accuracy: {round(accuracy_score(y_test, y_pred_rf), 2)}\n'
		   f'XGBoost Accuracy: { round(accuracy_score(y_test, y_pred_xgb), 2)}\n'
		   f'\n'
		   f'Random Forest F1: {round(f1_score(y_test, y_pred_rf, average="macro"), 2)}\n'
		   f'XGBoost F1: {round(f1_score(y_test, y_pred_xgb, average="macro"), 2)}\n'
		   f'\n'
		   f'Number of records: {len(y_train)}\n'
		   f'Random Forest Training Time: {round((end_fit_rf - start_fit_rf), 2)} s\n'
		   f'XGBoost Training Time: {round((end_fit_xgb - start_fit_xgb), 2)} s\n'
		   f'\n'
		   f'Number of records: {len(y_test)}\n'
		   f'Random Forest Inference Time: {round((end_predict_rf - start_predict_rf), 2)} s\n'
		   f'XGBoost Inference Time: {round((end_predict_xgb - start_predict_xgb), 2)} s')
print(content)

Random Forest Accuracy: 0.87
XGBoost Accuracy: 0.91

Random Forest F1: 0.7
XGBoost F1: 0.74

Number of records: 33585
Random Forest Training Time: 53.24 s
XGBoost Training Time: 1277.3 s

Number of records: 11195
Random Forest Inference Time: 1.32 s
XGBoost Inference Time: 0.23 s


## Save Model

In [35]:
def mkpath(path: str):
    if not os.path.exists(path):
        os.makedirs(path)

In [36]:
vectorizers = {
	"candidato": vectorizer_candidato,
	"vaga": vectorizer_vaga,
	"comentario": vectorizer_comentario
}

### Encoders

In [37]:
enc_path = "./train/"

mkpath(enc_path)

joblib.dump(label_encoders, enc_path + "label_encoders.gz")
joblib.dump(vectorizers, enc_path + "vectorizers.gz")
joblib.dump(scaler, enc_path + "standard_scaler.gz")

['./train/standard_scaler.gz']

### Random Forest

In [38]:
rf_path = "./train/RandomForest/"

mkpath(rf_path)

joblib.dump(rf, rf_path + "model.gz")

['./train/RandomForest/model.gz']

### XGBoost

In [39]:
xgb_path = "./train/XGBoost/"

mkpath(xgb_path)

joblib.dump(xgb, xgb_path + "model.gz")

['./train/XGBoost/model.gz']

## Update README

In [40]:
readme_path = "../README.md"

with open(readme_path, "r", encoding="utf-8") as file:
	reademe_content = file.read()

start_maker = "<!-- START_SCORE -->"
end_maker = "<!-- END_SCORE -->"
new_section = f"{start_maker}\n```\n{content}\n```\n{end_maker}"

if start_maker in reademe_content and end_maker in reademe_content:
	update = reademe_content.replace(
		reademe_content[reademe_content.find(start_maker):reademe_content.find(end_maker) + len(end_maker)],
		new_section
    )
else:
	update = reademe_content + "\n" + new_section

with open(readme_path, "w", encoding="utf-8") as file:
	file.write(update)

print("README updated successfully")

README updated successfully
